<a href="https://colab.research.google.com/github/cipz/AlgoritmiAvanzati/blob/master/Homework2/exactHeldKarp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Held and Karp algorithm

In [409]:
!pip install ipython-autotime

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 4.3 s


In [416]:
lines = open("ulysses22.tsp", "r").readlines()
print(lines)
n = int(lines[3].split()[1]) #.split()[0] # extract number of vertexes
t = lines[4].split()[1]
coordinates = lines[6]

'''for line in lines:
  if line.split()[0] == 'DIMENSION:':
    n = line.split()[0]
  else if line.split()[0] == 'EDGE_WEIGHT_TYPE:'
    t = line.split()[0]
  print()'''

V = [] # array of vertexes/coordinates
for i in range(7, 7 + n):
  V.append((int(lines[i].split()[0]) - 1, [float(lines[i].split()[1]), float(lines[i].split()[2])])) #(i, [x_value, y_value])

import copy

print(V)

x1 = 38.24
y1 = 20.42
c1 = []
c1.append(x1)
c1.append(y1)
x2 = 38.42
y2 = 13.11


for v in V:
  print(weight(c1, v[1], t) + 6576)

['NAME: ulysses22.tsp\n', 'TYPE: TSP\n', 'COMMENT: Odyssey of Ulysses (Groetschel/Padberg)\n', 'DIMENSION: 22\n', 'EDGE_WEIGHT_TYPE: GEO\n', 'DISPLAY_DATA_TYPE: COORD_DISPLAY\n', 'NODE_COORD_SECTION\n', ' 1 38.24 20.42\n', ' 2 39.57 26.15\n', ' 3 40.56 25.32\n', ' 4 36.26 23.12\n', ' 5 33.48 10.54\n', ' 6 37.56 12.19\n', ' 7 38.42 13.11\n', ' 8 37.52 20.44\n', ' 9 41.23 9.10\n', ' 10 41.17 13.05\n', ' 11 36.08 -5.21\n', ' 12 38.47 15.13\n', ' 13 38.15 15.35\n', ' 14 37.51 15.17\n', ' 15 35.49 14.32\n', ' 16 39.36 19.56\n', ' 17 38.09 24.36\n', ' 18 36.09 23.00\n', ' 19 40.44 13.57\n', ' 20 40.33 14.15\n', ' 21 40.37 14.23\n', ' 22 37.57 22.56\n', 'EOF\n', '\n']
[(0, [38.24, 20.42]), (1, [39.57, 26.15]), (2, [40.56, 25.32]), (3, [36.26, 23.12]), (4, [33.48, 10.54]), (5, [37.56, 12.19]), (6, [38.42, 13.11]), (7, [37.52, 20.44]), (8, [41.23, 9.1]), (9, [41.17, 13.05]), (10, [36.08, -5.21]), (11, [38.47, 15.13]), (12, [38.15, 15.35]), (13, [37.51, 15.17]), (14, [35.49, 14.32]), (15, [39.36

In [411]:
# global values

subsets = []
subsets.append(V)
phi = [[None] * 10000] * 100000
d = [[None] * 1000000] * 1000000

def get_subset_index (S):
  cont = 0
  for s in subsets:
    if s == S:
      return cont
    else:
      cont += 1
  return False

print(get_subset_index(V))

0
time: 23.2 ms


In [412]:
import math
# u, v: vertexes/coordinates
# t: WEIGHT_TYPE
def weight (u, v, t):
  if t == "EUC_2D":
    return round(math.sqrt(sum([(a - b) ** 2 for a, b in zip(u, v)])))
  else:
    PI = 3.141592
    deg_xu = int(u[0])
    min_xu = u[0] - deg_xu
    rad_xu = PI * (deg_xu + 5.0 * min_xu/ 3.0) / 180.0

    deg_yu = int(u[1])
    min_yu = u[1] - deg_yu
    rad_yu = PI * (deg_yu + 5.0 * min_yu/ 3.0) / 180.0

    deg_xv = int(v[0])
    min_xv = v[0] - deg_xv
    rad_xv= PI * (deg_xv + 5.0 * min_xv/ 3.0) / 180.0

    deg_yv = int(v[1])
    min_yv = v[1]- deg_yv
    rad_yv = PI * (deg_yv + 5.0 * min_yv/ 3.0) / 180.0

    RRR = 6378.388
    q1 = math.cos(rad_yu - rad_yv)
    q2 = math.cos(rad_xu - rad_xv)
    q3 = math.cos(rad_xu + rad_xv)
    return (int) (RRR * math.acos(0.5 * ((1.0 + q1) * q2 - (1.0 - q1) * q3)) + 1.0)

time: 19.8 ms


In [413]:
def HK_VISIT (v, S): # v: arrival vertex of S, S: subset of vertexes 
  S_index = get_subset_index(S)
  #print(S_index)
  if (len(S) == 1) & (S[0][0] == v): # Caso base: la soluzione è il peso dell’arco {v, 0}
    #print("1 case")
    return weight(V[v][1], V[0][1], t)
  elif d[v][S_index] is not None: # Distanza già calcolata, ritorna il valore memorizzato
    #print("2 case")
    return d[v][S_index] 
  else:  # Caso ricorsivo: trova il minimo tra tutti i sottocammini
    #print("3 case")
    mindist = math.inf
    minprec = None
    temp = copy.deepcopy(S)
    temp = [i for i in temp if i[0] != v] # S \ {v}
    for u in temp:
      dist = HK_VISIT(u[0], temp)
      subsets.append(temp)
      if (dist + weight(u[1], V[v][1], t)) < mindist:
        mindist = dist + weight(u[1], V[v][1], t)
        minprec = u
    d[v][S_index] = mindist
    phi[v][S_index] = minprec
    return mindist

time: 28.6 ms


In [414]:
def HK_TSP (G):
  G = (V, E) # build graph
  d = 0 # initialize matrix d
  phi = 0 # initialize matrix phi
  return HK_VISIT(0, V)

time: 2.58 ms


In [0]:
print(HK_VISIT(0, V))
print(len(subsets))
import numpy as np
#print(np.matrix(subsets))
#print(np.matrix(d))